In [ ]:
import pandas as pd
import numpy as np
import unidecode
import math
from shapely import wkb
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

## Loading data

In [ ]:
otodom = pd.read_csv("otodom.csv")
otodom["domain"] = "otodom"
print("otodom:", otodom.shape)
morizon = pd.read_csv("morizon.csv")
morizon["domain"] = "morizon"
print("morizon:", morizon.shape)
gratka = pd.read_csv("gratka.csv")
gratka["domain"] = "gratka"
print("gratka:", gratka.shape)
domiporta = pd.read_csv("domiporta.csv")
domiporta["domain"] = "domiporta"
print("domiporta:", domiporta.shape)

domains = [otodom, morizon, gratka, domiporta]

In [ ]:
cities = {
    'warszawa':'Warszawa',
    'krakow':'Kraków',
    'poznan':'Poznań',
    'wroclaw':'Wrocław',
    'gdansk':'Gdańsk',
    'gdynia':'Gdynia',
    'szczecin':'Szczecin',
    'bialystok':'Białystok',
    'katowice':'Katowice',
    'lublin':'Lublin',
    'lodz':'Łódź',
    'bydgoszcz':'Bydgoszcz',
}
districts = {
    'warszawa':{
        'mokotow':'Mokotów',
        'praga-poludnie':'Praga-Południe',
        'ursynow':'Ursynów',
        'wola':'Wola',
        'bielany':'Bielany',
        'targowek':'Targówek',
        'bemowo':'Bemowo',
        'srodmiescie':'Śródmieście',
        'bialoleka':'Białołęka',
        'wawer':'Wawer',
        'praga-polnoc':'Praga-Północ',
        'wlochy':'Włochy',
        'wilanow':'Wilanów',
        'rembertow':'Rembertów',
        'wesola':'Wesoła',
        'ochota':'Ochota',
        'ursus':'Ursus',
        'zoliborz':'Żoliborz',
        'praga':'Praga'
    },
    'krakow':{
        'stare-miasto':'Dzielnica I Stare Miasto',
        'grzegorzki':'Dzielnica II Grzegórzki',
        'pradnik-czerwony':'Dzielnica III Prądnik Czerwony',
        'pradnik-bialy':'Dzielnica IV Prądnik Biały',
        'pradnik':'Prądnik',
        'krowodrza':'Dzielnica V Krowodrza',
        'bronowice':'Dzielnica VI Bronowice',
        'zwierzyniec':'Dzielnica VII Zwierzyniec',
        'debniki':'Dzielnica VIII Dębniki',
        'lagiewniki-borek-falecki':'Dzielnica IX Łagiewniki-Borek Fałęcki',
        'swoszowice':'Dzielnica X Swoszowice',
        'podgorze-duchackie':'Dzielnica XI Podgórze Duchackie',
        'prokocim':'Dzielnica XII Bieżanów-Prokocim',
        'podgorze':'Dzielnica XIII Podgórze',
        'czyzyny':'Dzielnica XIV Czyżyny',
        'mistrzejowice':'Dzielnica XV Mistrzejowice',
        'bienczyce':'Dzielnica XVI Bieńczyce',
        'wzgorza-krzeslawickie':'Dzielnica XVII Wzgórza Krzesławickie',
        'nowa-huta':'Dzielnica XVIII Nowa Huta',
    },
    'poznan':{
        'antoninek-zieliniec-kobylepole':'Antoninek-Zieliniec-Kobylepole',
        'chartowo':'Chartowo',
        'fabianowo-kotowo':'Fabianowo-Kotowo',
        'glowna':'Główna',
        'gluszyna':'Głuszyna',
        'gorczyn':'Górczyn',
        'grunwald-poludnie':'Grunwald Południe',
        'grunwald-polnoc':'Grunwald Północ',
        'jana-iii-sobieskiego-i-marysienki':'Jana III Sobieskiego i Marysieńki',
        'jezyce':'Jeżyce',
        'junikowo':'Junikowo',
        'kiekrz':'Kiekrz',
        'krzesiny-pokrzywno-garaszewo':'Krzesiny-Pokrzywno-Garaszewo',
        'smochowice':'Krzyżowniki-Smochowice',
        'kwiatowe':'Kwiatowe',
        'lawica':'Ławica',
        'morasko-radojewo':'Morasko-Radojewo',
        'naramowice':'Naramowice',
        'nowe-winogrady-poludnie':'Nowe Winogrady Południe',
        'nowe-winogrady-polnoc':'Nowe Winogrady Północ',
        'nowe-winogrady-wschod':'Nowe Winogrady Wschód',
        'ogrody':'Ogrody',
        'ostrow-tumski-srodka-zawady-komandoria':'Ostrów Tumski-Śródka-Zawady-Komandoria',
        'piatkowo':'Piątkowo',
        'podolany':'Podolany',
        'rataje':'Rataje',
        'solacz':'Sołacz',
        'stare-miasto':'Stare Miasto',
        'stare-winogrady':'Stare Winogrady',
        'staroleka-minikowo-marlewo':'Starołęka-Minikowo-Marlewo',
        'stary-grunwald':'Stary Grunwald',
        'strzeszyn':'Strzeszyn',
        'szczepankowo-splawie-krzesinki':'Szczepankowo-Spławie-Krzesinki',
        'sw-lazarz':'Św. Łazarz',
        'swierczewo':'Świerczewo',
        'umultowo':'Umultowo',
        'warszawskie-pomet-maltanskie':'Warszawskie-Pomet-Maltańskie',
        'wilda':'Wilda',
        'winiary':'Winiary',
        'wola':'Wola',
        'zielony-debiec':'Zielony Dębiec',
    },
    'wroclaw':{
        'bienkowice':'Bieńkowice',
        'biskupin-sepolno-dabie-bartoszowice':'Biskupin - Sępolno - Dąbie - Bartoszowice',
        'borek':'Borek',
        'brochow':'Brochów',
        'gaj':'Gaj',
        'gajowice':'Gajowice',
        'gadow-popowice-pld':'Gądów - Popowice Płd.',
        'grabiszyn-grabiszynek':'Grabiszyn - Grabiszynek',
        'huby':'Huby',
        'jagodno':'Jagodno',
        'jerzmanowo-jarnoltow-strachowice-osiniec':'Jerzmanowo - Jarnołtów - Strachowice - Osiniec',
        'karlowice-rozanka':'Karłowice - Różanka',
        'klecina':'Klecina',
        'kleczkow':'Kleczków',
        'kowale':'Kowale',
        'krzyki':'Krzyki - Partynice',
        'ksieze':'Księże',
        'kuzniki':'Kuźniki',
        'lesnica':'Leśnica',
        'lipa-piotrowska':'Lipa Piotrowska',
        'maslice':'Maślice',
        'muchobor-maly':'Muchobór Mały',
        'muchobor-wielki':'Muchobór Wielki',
        'nadodrze':'Nadodrze',
        'nowy-dwor':'Nowy Dwór',
        'olbin':'Ołbin',
        'oltaszyn':'Ołtaszyn',
        'oporow':'Oporów',
        'osobowice-redzin':'Osobowice - Rędzin',
        'pawlowice':'Pawłowice',
        'pilczyce-kozanow-popowice-pln':'Pilczyce - Kozanów - Popowice Płn.',
        'plac-grunwaldzki':'Plac Grunwaldzki',
        'polanowice-poswietne-ligota':'Polanowice - Poświętne - Ligota',
        'powstancow-slaskich':'Powstańców Śląskich',
        'pracze-odrzanskie':'Pracze Odrzańskie',
        'przedmiescie-olawskie':'Przedmieście Oławskie',
        'przedmiescie-swidnickie':'Przedmieście Świdnickie',
        'psie-pole-zawidawie':'Psie Pole - Zawidawie',
        'soltysowice':'Sołtysowice',
        'stare-miasto':'Stare Miasto',
        'strachocin-swojczyce-wojnow':'Strachocin - Swojczyce - Wojnów',
        'szczepin':'Szczepin',
        'swiniary':'Świniary',
        'tarnogaj':'Tarnogaj',
        'widawa':'Widawa',
        'wojszyce':'Wojszyce',
        'zacisze-zalesie-szczytniki':'Zacisze - Zalesie - Szczytniki',
        'zerniki':'Żerniki',
    },
    'gdansk':{
        'krakowiec-gorki-zachodnie':'Krakowiec-Górki Zachodnie',
        'aniolki':'Aniołki',
        'zaspa-mlyniec':'Zaspa-Młyniec',
        'brzezno':'Brzeźno',
        'letnica':'Letnica',
        'matarnia':'Matarnia',
        'bretowo':'Brętowo',
        'chelm':'Chełm',
        'jasien':'Jasień',
        'kokoszki':'Kokoszki',
        'mlyniska':'Młyniska',
        'nowy-port':'Nowy Port',
        'oliwa':'Oliwa',
        'strzyza':'Strzyża',
        'olszynka':'Olszynka',
        'orunia-sw-wojciech-lipce':'Orunia-Św. Wojciech-Lipce',
        'osowa':'Osowa',
        'piecki-migowo':'Piecki-Migowo',
        'przerobka':'Przeróbka',
        'przymorze-male':'Przymorze Małe',
        'przymorze-wielkie':'Przymorze Wielkie',
        'rudniki':'Rudniki',
        'siedlce':'Siedlce',
        'stogi':'Stogi',
        'suchanino':'Suchanino',
        'ujescisko-lostowice':'Ujeścisko-Łostowice',
        'vii-dwor':'VII Dwór',
        'wzgorze-mickiewicza':'Wzgórze Mickiewicza',
        'wrzeszcz-dolny':'Wrzeszcz Dolny',
        'wrzeszcz-gorny':'Wrzeszcz Górny',
        'wyspa-sobieszewska':'Wyspa Sobieszewska',
        'zaspa-rozstaje':'Zaspa-Rozstaje',
        'srodmiescie':'Śródmieście',
        'zabianka-wejhera-jelitkowo-tysiaclecia':'Żabianka-Wejhera-Jelitkowo-Tysiąclecia',
    },
    'gdynia':{
        'obluze':'Obłuże',
        'maly-kack':'Mały Kack',
        'cisowa':'Cisowa',
        'leszczynki':'Leszczynki',
        'dabrowa':'Dąbrowa',
        'grabowek':'Grabówek',
        'chylonia':'Chylonia',
        'kamienna-gora':'Kamienna Góra',
        'oksywie':'Oksywie',
        'babie-doly':'Babie Doły',
        'orlowo':'Orłowo',
        'karwiny':'Karwiny',
        'wzgorze-sw-maksymiliana':'Wzgórze Św. Maksymiliana',
        'chwarzno-wiczlino':'Chwarzno-Wiczlino',
        'pogorze':'Pogórze',
        'pustki-cisowskie-demptowo':'Pustki Cisowskie-Demptowo',
        'dzialki-lesne':'Działki Leśne',
        'srodmiescie':'Śródmieście',
        'witomino':'Witomino Radiostacja',
        'wielki-kack':'Wielki Kack',
        'redlowo':'Redłowo',
        'witomino':'Witomino Leśniczówka',
    },
    'szczecin':{
        'polnoc':'Północ',
        'prawobrzeze':'Prawobrzeże',
        'srodmiescie':'Śródmieście',
        'zachod':'Zachód',
    },
    'bialystok':{
        'centrum':'Centrum',
        'bialostoczek':'Białostoczek',
        'sienkiewicza':'Sienkiewicza',
        'bojary':'Bojary',
        'piaski':'Piaski',
        'przydworcowe':'Przydworcowe',
        'mlodych':'Młodych',
        'antoniuk':'Antoniuk',
        'jaroszowka':'Jaroszówka',
        'wygoda':'Wygoda',
        'piasta-i':'Piasta I',
        'piasta-ii':'Piasta II',
        'skorupy':'Skorupy',
        'mickiewicza':'Mickiewicza',
        'dojlidy':'Dojlidy',
        'bema':'Bema',
        'kawaleryjskie':'Kawaleryjskie',
        'nowe-miasto':'Nowe Miasto',
        'zielone-wzgorza':'Zielone Wzgórza',
        'starosielce':'Starosielce',
        'sloneczny-stok':'Słoneczny Stok',
        'lesna-dolina':'Leśna Dolina',
        'wysoki-stoczek':'Wysoki Stoczek',
        'dziesieciny-i':'Dziesięciny I',
        'dziesieciny-ii':'Dziesięciny II',
        'bacieczki':'Bacieczki',
        'zawady':'Zawady',
    },
    'katowice':{
        'bogucice':'Bogucice',
        'brynow-osiedle-zgrzebnioka':'Brynów-Osiedle Zgrzebnioka',
        'dab':'Dąb',
        'dabrowka-mala':'Dąbrówka Mała',
        'giszowiec':'Giszowiec',
        'janow-nikiszowiec':'Janów-Nikiszowiec',
        'kostuchna':'Kostuchna',
        'koszutka':'Koszutka',
        'ligota-panewniki':'Ligota-Panewniki',
        'murcki':'Murcki',
        'osiedle-paderewskiego-muchowiec':'Osiedle Paderewskiego-Muchowiec',
        'osiedle-tysiaclecia':'Osiedle Tysiąclecia',
        'piotrowice-ochojec':'Piotrowice-Ochojec',
        'podlesie':'Podlesie',
        'burowiec':'Szopienice-Burowiec',
        'srodmiescie':'Śródmieście',
        'welnowiec-jozefowiec':'Wełnowiec-Józefowiec',
        'zaleska-halda-brynow':'Załęska Hałda-Brynów',
        'zaleze':'Załęże',
        'zarzecze':'Zarzecze',
        'zawodzie':'Zawodzie',
    },
    'lublin':{
        'srodmiescie':'Śródmieście',
        'weglin-polnocny':'Węglin Północny',
        'weglin-poludniowy':'Węglin Południowy',
        'zemborzyce':'Zemborzyce',
        'za-cukrownia':'Za Cukrownią',
        'tatary':'Tatary',
        'slawinek':'Sławinek',
        'wrotkow':'Wrotków',
        'wieniawa':'Wieniawa',
        'stare-miasto':'Stare Miasto',
        'rury':'Rury',
        'slawin':'Sławin',
        'szerokie':'Szerokie',
        'kalinowszczyzna':'Kalinowszczyzna',
        'hajdow-zadebie':'Hajdów-Zadębie',
        'ponikwoda':'Ponikwoda',
        'kosminek':'Kośminek',
        'dziesiata':'Dziesiąta',
        'czuby-polnocne':'Czuby Północne',
        'glusk':'Głusk',
        'felin':'Felin',
        'czechow-poludniowy':'Czechów Południowy',
        'bronowice':'Bronowice',
        'czuby-poludniowe':'Czuby Południowe',
        'czechow-polnocny':'Czechów Północny',
        'konstantynow':'Konstantynów',
        'abramowice':'Abramowice',
    },
    'lodz':{
        'polesie':'Polesie',
        'widzew':'Widzew',
        'srodmiescie':'Śródmieście',
        'baluty':'Bałuty',
        'gorna':'Górna',
    },
    'bydgoszcz':{},
}

## Removing duplicated urls/headers

### Removing duplicated urls

In [ ]:
for df in domains:
    print("-----------------")
    print(df.shape)
    df.drop_duplicates(subset=['url'], inplace=True)
    print(df.shape)

### Removing duplicated headers

In [ ]:
for df in domains:
    print("-----------------")
    print(df.shape)
    df.drop(df[df.url == "url"].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(df.shape)

## Add IDs to all observations

In [ ]:
start = 1
for df in domains:
    end = start+len(df)
    df.insert(0, "id", range(start, end))
    start=end
    print("-----------------")
    print(df.shape)
    print(df["id"].nunique())

In [ ]:
def remove_accents(a):
    return unidecode.unidecode(a)

## OTODOM cleaning

In [ ]:
otodom.head(3)

In [ ]:
otodom["deposit"] = otodom["deposit"].str.replace(r'[^0-9]', '') # 2 500 zł --> 2500

otodom["price"] = otodom["price"].str.replace(r'[^0-9]', '') # 2 500 zł --> 2500

otodom["year_built"] = otodom["year_built"].str.replace(r'[^0-9]', '') # to number

otodom["rooms"] = otodom["rooms"].str.replace(r'[^0-9]', '') # to number

otodom["floor"] = otodom["floor"].str.replace('> 10', '11') # > 10 --> 11
otodom["floor"] = otodom["floor"].str.replace('parter', '0') # > parter --> 0
otodom["floor"] = otodom["floor"].str.replace('suterena', '-1') # > suterena --> -1
otodom.loc[otodom["floor"] == "poddasze", 'floor'] = otodom.loc[otodom["floor"] == "poddasze", 'floors']  # poddasze --> floors
otodom["floor"] = otodom["floor"].str.replace(r'[^0-9]', '') # > to number

otodom["size"] = otodom["size"].str.replace(' m²', '') # 50 m² --> 50
otodom["size"] = otodom["size"].str.replace(',', '.') # 50,49 --> 50.49

# get city from address
otodom["city"] = otodom['address'].str.split(pat=",").str[0].str.lower().apply(remove_accents).replace(" ", "-")
# get district from address
otodom["district"] = otodom['address'].str.split(pat=",").str[1].str.lower().apply(remove_accents).replace(" ", "-")
# simplify description
otodom["description"] = otodom['description'].astype(str).str.lower().apply(remove_accents)

# features
otodom["features"] = otodom["features"].str.split(",")

otodom["balcony"] = otodom["features"].apply(lambda x: (("balkon" in x) or ("taras" in x)) if isinstance(x, list) else 0)
otodom["furnished"] = otodom["features"].apply(lambda x: ("meble" in x) if isinstance(x, list) else 0)
otodom["parking"] = otodom["features"].apply(lambda x: (("garaż/miejsce parkingowe" in x) or ("miejsce parkingowe" in x)) if isinstance(x, list) else 0)
otodom["elevator"] = otodom["features"].apply(lambda x: ("winda" in x) if isinstance(x, list) else 0)
otodom["separate_kitchen"] = otodom["features"].apply(lambda x: ("oddzielna kuchnia" in x) if isinstance(x, list) else 0)
otodom["storage"] = otodom["features"].apply(lambda x: (("piwnica" in x) or ("pom. użytkowe" in x)) if isinstance(x, list) else 0)
otodom["backyard"] = otodom["features"].apply(lambda x: ("ogródek" in x) if isinstance(x, list) else 0)
otodom["internet"] = otodom["features"].apply(lambda x: ("internet" in x) if isinstance(x, list) else 0)
otodom["tv"] = otodom["features"].apply(lambda x: ("telewizor" in x) if isinstance(x, list) else 0)
otodom["dishwasher"] = otodom["features"].apply(lambda x: ("zmywarka" in x) if isinstance(x, list) else 0)
otodom["washing_machine"] = otodom["features"].apply(lambda x: ("pralka" in x) if isinstance(x, list) else 0)

otodom["standard"] = otodom["status"].map(
    {'do zamieszkania': 'wysoki', 'do wykończenia': 'dobry', 'do remontu': 'niski'}
)

otodom["building_material"] = otodom["building_material"].map(
    {'beton komórkowy':'beton', 'pustak':'pustak', 'cegła':'cegla', 'beton':'beton', 'inne':'inny', 'wielka płyta':'wielka plyta', 'silikat':'silikat', 'żelbet':'inny', 'keramzyt':'inny', 'drewno':'inny'}
)


otodom.loc[otodom['building_type'].str.contains('dom wolnostojący', na=False), 'building_type'] = "inny"
otodom.loc[otodom['building_type'].str.contains('loft', na=False), 'building_type'] = "inny"
otodom.loc[otodom['building_type'].str.contains('plomba', na=False), 'building_type'] = "inny"
otodom.loc[otodom['building_type'].str.contains('inny', na=False), 'building_type'] = "inny"
otodom.loc[otodom['building_type'].str.contains('apartamentowiec', na=False), 'building_type'] = "apartamentowiec"
otodom.loc[otodom['building_type'].str.contains('blok', na=False), 'building_type'] = "blok"
otodom.loc[otodom['building_type'].str.contains('kamienica', na=False), 'building_type'] = "kamienica"

otodom.sample(3)

#### Categorical variables - unique values

In [ ]:
otodom_building_material = list(otodom.building_material.dropna().unique())
print(otodom_building_material)

otodom_building_type = list(otodom.building_type.dropna().unique())
print(otodom_building_type)

otodom_standard = list(otodom.standard.dropna().unique())
print(otodom_standard)

otodom_windows = list(otodom.windows.dropna().unique())
print(otodom_windows)

## MORIZON cleaning

In [ ]:
morizon.head(3)

In [ ]:
# simplify description
morizon["description"] = morizon['description'].astype(str).str.lower().apply(remove_accents)

morizon["balcony"] = morizon["balcony"].astype(str).apply(lambda x: (("Tak" in x) or ("m²" in x)) if x is not None else 0)
morizon["building_material"] = morizon["building_material"].astype(str).str.replace("\n","").str.lower().apply(remove_accents)
morizon["building_material"] = morizon["building_material"].apply(lambda x: "cegła" if "cegla" in x else x)
morizon["building_type"] = morizon["building_type"].astype(str).str.replace("\n","").str.lower().apply(remove_accents)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "blok" if "blok" in x else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "apartamentowiec" if "apartament" in x else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "dom wolnostojący" if (("dom" in x)or ("wolnostojący" in x)) else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "blok" if "budownictwo" in x else x)

morizon["deposit"] = morizon["deposit"].astype(str).str.replace(r'[^0-9]', '')
morizon["deposit"] = pd.to_numeric(morizon["deposit"],errors='coerce')

morizon["equipment"] = morizon["equipment"].astype(str).str.replace("\n","").str.replace(", ",",")
morizon["features"] = morizon["features"].astype(str).str.replace("\n","").str.replace(", ",",").str.replace(" ,",",")
morizon["features"] = morizon["equipment"] + "," + morizon["features"]
morizon["features"] = morizon["features"].str.lower().apply(remove_accents)
morizon["features"] = morizon["features"].str.split(",")

morizon["furnished"] = morizon["features"].apply(lambda x: (("meble" in x) and ("(nie)" not in x)) if isinstance(x, list) else 0)
morizon["parking"] = morizon["features"].apply(lambda x: ("parkin" in x) if isinstance(x, list) else 0)
morizon["elevator"] = morizon["features"].apply(lambda x: (("wind" in x) and ("brak" not in x)) if isinstance(x, list) else 0)
morizon["storage"] = morizon["features"].apply(lambda x: (("piwnica" in x) and ("(nie)" not in x)) if isinstance(x, list) else 0)
morizon["internet"] = morizon["features"].apply(lambda x: (("internet" in x) and ("brak" not in x)) if isinstance(x, list) else 0)
morizon["tv"] = morizon["features"].apply(lambda x: ("telewizor" in x) if isinstance(x, list) else 0)
morizon["washing_machine"] = morizon["features"].apply(lambda x: ("pralka" in x) if isinstance(x, list) else 0)

morizon["floor"] = morizon["floor"].astype(str).str.replace("\n","").str.replace(" ","").str.split('/').str[0]
morizon["floor"] = morizon["floor"].str.replace('parter', '0')

morizon["price"] = morizon["price"].str.replace(r'[^0-9]', '')
morizon["size"] = otodom["size"].str.replace(',', '.')

morizon["standard"] = morizon['standard'].astype(str).str.replace("\n","").str.lower().apply(remove_accents)

morizon["title"] = morizon['title'].astype(str).str.replace("\n","").str.replace(", ",",").str.replace(" ,",",").str.replace(",,",",")
morizon["title"] = morizon['title'].str.lower().apply(remove_accents).replace(" ", "-")

cities_list = list(cities.keys())
morizon["city"] = morizon['title'].apply(lambda x: next((city for city in cities_list if city in x), None))

districts_list = []
for city in districts.keys():
    districts_list += list(districts[city].keys())
morizon["district"] = morizon['title'].apply(lambda x: next((district for district in districts_list if district in x.replace(" ", "-")), None))

morizon["standard"] = morizon["standard"].astype(str).str.replace("\n",",").str.lower().apply(remove_accents)
morizon["standard"] = morizon["standard"].apply(
    lambda x: "wysoki" if (
        ("wysok" in x) or 
        ("luksus" in x) or 
        ("ideal" in x) or 
        ("bardzo" in x) or 
        ("komfo" in x) or 
        ("podwyzsz" in x) or 
        ("general" in x) or
        ("wykonczo" in x)
    ) else x
)
morizon["standard"] = morizon["standard"].apply(
    lambda x: "dobry" if (
        ("dobr" in x) or 
        ("wprowadze" in x) or 
        ("zamieszka" in x) or 
        ("bardzo" in x) or 
        ("komfort" in x) or 
        ("po remo" in x) or 
        ("po-remo" in x) or
        ("po odsw" in x) or
        ("sredni" in x) or
        ("odnowio" in x) or
        ("do wykoncz" in x) or 
        ("dopasowa" in x) or
        ("normaln" in x)
    ) else x
)
morizon["standard"] = morizon["standard"].apply(
    lambda x: "niski" if (
        ("niski" in x) or
        ("do odnow" in x) or 
        ("do odswie" in x) or 
        ("do remo" in x) or 
        ("remont" in x) or 
        ("dewelop" in x) 
    ) else x
)
morizon["standard"] = morizon["standard"].apply(lambda x: np.nan if x not in ["wysoki", "dobry", "niski"] else x)

morizon["building_material"] = morizon["building_material"].astype(str).str.replace("\n",",").str.replace(" ","").str.replace("-","").str.lower().apply(remove_accents)
morizon["building_material"] = morizon["building_material"].apply(lambda x: "beton" if ("beton" in x) else x)
morizon["building_material"] = morizon["building_material"].apply(lambda x: "cegla" if ("cegla" in x) else x)
morizon["building_material"] = morizon["building_material"].apply(lambda x: "pustak" if ("pustak" in x) else x)
morizon["building_material"] = morizon["building_material"].apply(lambda x: "wielka plyta" if ("plyta" in x) else x)
morizon["building_material"] = morizon["building_material"].apply(lambda x: "silikat" if ("silikat" in x) else x)
morizon["building_material"] = morizon["building_material"].apply(
    lambda x: "inny" if x not in ["beton", "cegla", "pustak", "wielka plyta", "silikat", "nan"] else x
)
morizon["building_material"] = morizon["building_material"].apply(
    lambda x: np.nan if x=='nan' else x
)

morizon["building_type"] = morizon["building_type"].astype(str).str.replace("\n",",").str.replace(" ","").str.replace("-","").str.lower().apply(remove_accents)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "szeregowiec" if ("wielo" in x) else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "szeregowiec" if ("szeregowiec" in x) else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "blok" if ("blok" in x) else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "apartamentowiec" if ("apartam" in x) else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "kamienica" if ("kamien" in x) else x)
morizon["building_type"] = morizon["building_type"].apply(lambda x: "inny" if ("inn" in x) else x)
morizon["building_type"] = morizon["building_type"].apply(
    lambda x: "inny" if x not in ["blok", "apartamentowiec", "kamienica", "szeregowiec", "silikat"] else x
)

morizon["windows"] = morizon["windows"].astype(str).str.replace("\n",",").str.replace(" ","").str.replace("-","").str.lower().apply(remove_accents)
morizon["windows"] = morizon["windows"].apply(lambda x: "plastikowe" if (("plastik" in x) or ("pcv" in x)) else x)
morizon["windows"] = morizon["windows"].apply(lambda x: "drewniane" if ("drew" in x) else x)
morizon["windows"] = morizon["windows"].apply(lambda x: "aluminiowe" if ("alumi" in x) else x)
morizon["windows"] = morizon["windows"].apply(
    lambda x: np.nan if x not in ["plastikowe","drewniane","aluminiowe"] else x
)

morizon.sample(3)

In [ ]:
morizon_building_material = list(morizon.building_material.dropna().unique())
print(morizon_building_material)

morizon_building_type = list(morizon.building_type.dropna().unique())
print(morizon_building_type)

morizon_standard = list(morizon.standard.dropna().unique())
print(morizon_standard)

morizon_windows = list(morizon.windows.dropna().unique())
print(morizon_windows)

## GRATKA cleaning

In [ ]:
gratka.head(3)

In [ ]:
gratka["address"] = gratka["address"].astype(str).str.replace("\n","").str.replace("  ","").str.lower().apply(remove_accents).replace(" ", "-")
gratka["address"] = gratka["address"].replace(", ", ",").replace(" ,", ",")

cities_list = list(cities.keys())
gratka["city"] = gratka['address'].apply(lambda x: next((city for city in cities_list if city in x), None))

districts_list = []
for city in districts.keys():
    districts_list += list(districts[city].keys())
gratka["district"] = gratka['address'].apply(lambda x: next((district for district in districts_list if district in x.replace(" ", "-")), None))

gratka["description"] = gratka['description'].astype(str).str.replace("\n","").str.lower().apply(remove_accents)
gratka["features"] = gratka["features"].astype(str).str.replace("\n",",").str.lower().apply(remove_accents)
gratka["features"] = gratka["features"].str.replace(",,",",").str.replace(",,",",").str.replace(" ","").str.split(",")
gratka["balcony"] = gratka["features"].apply(lambda x: ("balkon" in x) if isinstance(x, list) else 0)
gratka["internet"] = gratka["features"].apply(lambda x: "internet" in x if isinstance(x, list) else 0)
gratka["storage"] = gratka["features"].apply(lambda x: (("piwnica" in x) or ("powierzchnia" in x) or ("komorka" in x)) if isinstance(x, list) else 0)
gratka["parking"] = gratka["parking"].apply(lambda x: False if math.isnan(x) else True)
gratka["tv"] = gratka["features"].apply(lambda x: ("telewiz" in x) if isinstance(x, list) else 0)

gratka["floor"] = gratka["floor"].str.replace('niski parter', '0')
gratka["floor"] = gratka["floor"].str.replace('parter', '0')
gratka["floor"] = gratka["floor"].str.replace('powyżej 30', '31')
gratka.loc[otodom["floor"] == "poddasze", 'floor'] = gratka.loc[otodom["floor"] == "poddasze", 'floors']
gratka = gratka.drop(gratka[gratka["floor"].str.len() > 3].index)

gratka.loc[gratka['floors'].str.contains('0 ', na=False), 'floors'] = "0"
gratka["floors"] = gratka["floors"].str.replace('powyżej 30', '31')

gratka["separate_kitchen"] = gratka["kitchen"].astype(str).apply(lambda x: True if x=="oddzielna" else False)

gratka["price"] = gratka["price"].str.replace(r'[^0-9]', '')
gratka.loc[gratka['rooms'].str.contains('więcej', na=False), 'rooms'] = "9"
gratka["size"] = gratka['size'].str.split(pat=",").str[0]
gratka["size"] = gratka['size'].str.split(pat=" ").str[0]

gratka.loc[gratka['standard'].str.contains('bardzo dobry', na=False), 'standard'] = "wysoki"
gratka.loc[gratka['standard'].str.contains('wysoki', na=False), 'standard'] = "wysoki"
gratka.loc[gratka['standard'].str.contains('dobry', na=False), 'standard'] = "dobry"
gratka.loc[gratka['standard'].str.contains('remont', na=False), 'standard'] = "niski"
gratka.loc[gratka['standard'].str.contains('niski', na=False), 'standard'] = "niski"
gratka.loc[gratka['standard'].str.contains('odświeżenia', na=False), 'standard'] = "niski"
gratka.loc[gratka['standard'].str.contains('remoncie', na=False), 'standard'] = "wysoki"
gratka.loc[gratka['standard'].str.contains('\n', na=False), 'standard'] = "dobry"
gratka.loc[gratka['standard'].str.contains('nowa', na=False), 'standard'] = "dobry"
gratka.loc[gratka['standard'].str.contains('wymianie', na=False), 'standard'] = "dobry"
gratka.loc[gratka['standard'].str.contains('częściowo', na=False), 'standard'] = "niski"

gratka["building_material"] = gratka["building_material"].map(
    {'beton':'beton', 'płyta':'wielka plyta', 'cegła':'cegla', 'rama H':'inny', 'silikat':'silikat', 'pustak':'pustak', 'inne':'inny', 'konstrukcja stalowa':'inny', 'drewno':'inny', 'kamień':'inny'}
)

gratka.loc[gratka['building_type'].str.contains('dom wielorodzinny/szeregowiec', na=False), 'building_type'] = "szeregowiec"
gratka.loc[gratka['building_type'].str.contains('loft', na=False), 'building_type'] = "inny"
gratka.loc[gratka['building_type'].str.contains('plomba', na=False), 'building_type'] = "inny"
gratka.loc[gratka['building_type'].str.contains('inny', na=False), 'building_type'] = "inny"
gratka.loc[gratka['building_type'].str.contains('apartamentowiec', na=False), 'building_type'] = "apartamentowiec"
gratka.loc[gratka['building_type'].str.contains('blok', na=False), 'building_type'] = "blok"
gratka.loc[gratka['building_type'].str.contains('kamienica', na=False), 'building_type'] = "kamienica"


gratka.sample(3)

In [ ]:
gratka_building_material = list(gratka.building_material.dropna().unique())
print(gratka_building_material)

gratka_building_type = list(gratka.building_type.dropna().unique())
print(gratka_building_type)

gratka_standard = list(gratka.standard.dropna().unique())
print(gratka_standard)

## DOMIPORTA cleaning

In [ ]:
domiporta.sample(3)

In [ ]:
domiporta["address"] = domiporta["address"].astype(str).str.replace("\n ","").str.replace("  ","").str.lower().apply(remove_accents).replace(" ", "-")
domiporta["address"] = domiporta["address"].replace(", ", ",").replace(" ,", ",")

cities_list = list(cities.keys())
domiporta["city"] = domiporta['address'].apply(lambda x: next((city for city in cities_list if city in x), None))

districts_list = []
for city in districts.keys():
    districts_list += list(districts[city].keys())
domiporta["district"] = domiporta['address'].apply(lambda x: next((district for district in districts_list if district in x.replace(" ", "-")), None))

domiporta["description"] = domiporta['description'].astype(str).str.replace("\n","").str.lower().apply(remove_accents)

domiporta["features"] = domiporta["features"].astype(str).str.replace("\n",",").str.lower().apply(remove_accents)
domiporta["features"] = domiporta["features"].str.replace(",,",",").str.replace(",,",",").str.replace(" ","").str.split(",")
domiporta.features.explode().unique()
domiporta["balcony"] = domiporta["features"].apply(lambda x: (("balkon" in x) or ("taras" in x)) if isinstance(x, list) else 0)
domiporta["furnished"] = domiporta["features"].apply(lambda x: ("meble" in x) if isinstance(x, list) else 0)
domiporta["parking"] = domiporta["features"].apply(lambda x: (("parking" in x) or ("garaz" in x)) if isinstance(x, list) else 0)
domiporta["elevator"] = domiporta["features"].apply(lambda x: ("winda" in x) if isinstance(x, list) else 0)
domiporta["separate_kitchen"] = domiporta["features"].apply(lambda x: ("kuchnia" in x) if isinstance(x, list) else 0)
domiporta["dishwasher"] = domiporta["features"].apply(lambda x: ("zmywarka" in x) if isinstance(x, list) else 0)
domiporta["washing_machine"] = domiporta["features"].apply(lambda x: ("pralka" in x) if isinstance(x, list) else 0)
domiporta["storage"] = domiporta["features"].apply(lambda x: (("piwnica" in x) or ("pom. użytkowe" in x)) if isinstance(x, list) else 0)
domiporta["backyard"] = domiporta["features"].apply(lambda x: ("ogródek" in x) if isinstance(x, list) else 0)
domiporta["internet"] = domiporta["features"].apply(lambda x: ("internet" in x) if isinstance(x, list) else 0)
domiporta["tv"] = domiporta["features"].apply(lambda x: ("telewiz" in x) if isinstance(x, list) else 0)

domiporta["floors"] = domiporta["floors"].astype(str).str.replace('parter', '0').str.replace(' ', '').str.split('/')
domiporta["floor"] = domiporta["floors"].apply(lambda x: x[0])
domiporta["floor"] = domiporta["floor"].str.replace('parter', '0')
domiporta.loc[domiporta["floor"]=="-", "floor"] = None

domiporta["floors"] = domiporta["floors"].apply(lambda x: x[1].replace('-', '') if len(x)>1 else None)

domiporta["latitude"] = domiporta["latitude"].str.replace(",", ".").astype(float)
domiporta["longitude"] = domiporta["longitude"].str.replace(",", ".").astype(float)

domiporta["size"] = domiporta["floors"].astype(str).str.replace(' m', '').str.replace(',', '.')
domiporta["size"] = domiporta["size"].fillna(value={"size":np.nan})
domiporta.loc[domiporta["size"].isin(['None','']), "size"] = np.nan
domiporta["size"] = domiporta["size"].astype(float)

domiporta["building_material"] = domiporta["building_material"].map(
    {'beton':'beton', 'inny':'inny', 'cegła':'cegla', 'rama h':'inny', 'pustak':'pustak', 'wielka płyta':'wielka plyta', 'silikat':'silikat', 'ytong':'inny'}
)

domiporta.loc[domiporta["building_type"] == 'dom wielorodzinny', "building_type"] = 'szeregowiec'

domiporta.sample(3)

In [ ]:
domiporta_building_material = list(domiporta.building_material.dropna().unique())
print(domiporta_building_material)

domiporta_building_type = list(domiporta.building_type.dropna().unique())
print(domiporta_building_type)

## Merging dataframes

#### Columns

In [ ]:
columns = [
    'id',
    'building_material',
    'building_type',
    'description',
    'features',
    'floor',
    'floors',
    'latitude',
    'longitude',
    'price',
    'rooms',
    'size',
    'title',
    'url',
    'windows',
    'year_built',
    'city',
    'district',
    'balcony',
    'furnished',
    'parking',
    'elevator',
    'separate_kitchen',
    'storage',
    'internet',
    'tv',
    'standard',
    'domain',
    'coordinates',
]


In [ ]:
domains = [otodom, morizon, gratka, domiporta]
for domain in domains:
    print(domain.shape[0], "rows")
print("+ --------")
df = pd.concat(domains, ignore_index=True)
print(df.shape[0], "rows")

In [ ]:
df = df.replace('nan', np.nan)
df["description"] = df["description"].astype(str).replace(",","")
df["rooms"] = df["rooms"].astype('float')
df["rooms"] = pd.Series(df["rooms"].tolist(), dtype="Int64")
df['size'] = np.floor(pd.to_numeric(df['size'], errors='coerce')).astype('Int64')
df['floor'] = np.floor(pd.to_numeric(df['floor'], errors='coerce')).astype('Int64')
df['floors'] = np.floor(pd.to_numeric(df['floors'], errors='coerce')).astype('Int64')
df['price'] = np.floor(pd.to_numeric(df['price'], errors='coerce')).astype('Int64')
df['year_built'] = np.floor(pd.to_numeric(df['year_built'], errors='coerce')).astype('Int64')
df['id'] = np.floor(pd.to_numeric(df['id'], errors='coerce')).astype('Int64')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df["coordinates"] = list(zip(df["latitude"], df["longitude"]))
df = df.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)

cat_vars = [
    'building_material',
    'building_type',
    'windows',
    'balcony',
    'furnished',
    'parking',
    'elevator',
    'separate_kitchen',
    'storage',
    'internet',
    'tv',
    'standard',
    'domain',
]

for x in cat_vars:
    print(x,": ", df[x].value_counts(dropna=False))
    print("--------------------------")

## Districts

In [ ]:
boundaries = pd.read_csv("districts_boundaries.csv")
boundaries.sample(3)

In [ ]:
def get_map_shapes(df_shapes):
    df = pd.DataFrame(
        columns=["longitude", "latitude", "city_id", "district"]
    )
    for i in range(len(df_shapes)):
        shape = wkb.loads(str(df_shapes.at[i, "boundaries"]), hex=True)
        df_coord = pd.DataFrame(
            list(shape.exterior.coords), columns=["longitude", "latitude"]
        )
        df_coord["district"] = df_shapes.at[i, "slug"]
        df_coord["city_id"] = df_shapes.at[i, "city_id"]
        df = pd.concat([df, df_coord], ignore_index=True)
    return df 

boundaries = get_map_shapes(boundaries)
boundaries["coordinates"] = list(zip(boundaries["latitude"], boundaries["longitude"]))
boundaries.sample(3)

In [ ]:
boundaries = boundaries.groupby(['district','city_id'])['coordinates'].apply(list).reset_index(name='polygon')
boundaries.sample(3)

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from tqdm import tqdm

cities = {
    1:"warszawa",
    6:"krakow",
    7:"poznan",
    5:"wroclaw",
    2:"gdansk",
    3:"gdynia",
    9:"szczecin",
    13:"bialystok",
    14:"katowice",
    12:"lublin",
    8:"lodz",
    10:"bydgoszcz",
}
for i in tqdm(range(len(df))):
    point = Point(df.at[i, 'coordinates'])
    for j in range(len(boundaries)):
        polygon = Polygon(boundaries.at[j,'polygon'])
        if polygon.contains(point):
            df.at[i, 'district'] = boundaries.at[j,'district']
            df.at[i, 'city'] = cities[boundaries.at[j,'city_id']]
            break

In [ ]:
df.sample(10)

In [ ]:
df = df[columns]
df = df[df["city"].isin(list(cities.values()))]
df["district"] = df["district"].astype(str).str.replace(" ","")
df = df[df["district"] != "None"].reset_index(drop=True)

In [ ]:
df["description"]=df["description"].str.replace(",","")
df.columns

## MISSING VALUES

In [ ]:
df.elevator.value_counts(dropna=False)

In [ ]:
elevator_words = ["wind"]
elevator_stop_words = ["bez wind", "bez  wind", "brak wind", "brak  wind", "bezwind", "brakwind", "niemawind", "nie ma wind", "winda brak", "winda: brak", "winda: nie", "winda - nie"]
for i in tqdm(range(len(df))):
    if df.at[i,"elevator"] == False:
        df.at[i,"elevator"] = any(x in df.at[i,"description"] for x in elevator_words)
        df.at[i,"elevator"] = not any(x in df.at[i,"description"] for x in elevator_stop_words)

df[['elevator']] = df[['elevator']].fillna(False)
df.elevator.value_counts(dropna=False)

In [ ]:
df.furnished.value_counts(dropna=False)

In [ ]:
furnished_words = ["meble", "mebli", "umebl"]
furnished_stop_words = ["bez mebli", "nie jest ubebl", "brak mebl", "meble nie", "meble: nie", "meble brak"]
for i in tqdm(range(len(df))):
    if df.at[i,"furnished"] == False:
        df.at[i,"furnished"] = any(x in df.at[i,"description"] for x in furnished_words)
        df.at[i,"furnished"] = not any(x in df.at[i,"description"] for x in furnished_stop_words)
df[['furnished']] = df[['furnished']].fillna(False)
df.furnished.value_counts(dropna=False)

In [ ]:
df.separate_kitchen.value_counts(dropna=False)

In [ ]:
separate_kitchen_words = ["oddzielna kuchn", "osobna kuchn", "odrebna kuchn"]
separate_kitchen_stop_words = ["aneks", "otwarta kuch", "kuchnia polacz"]
for i in tqdm(range(len(df))):
    if df.at[i,"separate_kitchen"] == False:
        df.at[i,"separate_kitchen"] = any(x in df.at[i,"description"] for x in separate_kitchen_words)
        df.at[i,"separate_kitchen"] = not any(x in df.at[i,"description"] for x in separate_kitchen_stop_words)
df[['separate_kitchen']] = df[['separate_kitchen']].fillna(False)
df.separate_kitchen.value_counts(dropna=False)

In [ ]:
df.standard.value_counts(dropna=False)

In [ ]:
wysoki_words = ["luksus", "wysoki stan", "ekskluz", "wykonczon", "po remon"]
niski_words = ["do remon", "niski stan", "do wyko"]
for i in tqdm(range(len(df))):
    if df.at[i,"standard"] not in ['wysoki','dobry','niski']:
        df.at[i,"standard"] = 'wysoki' if any(x in df.at[i,"description"] for x in wysoki_words) else np.NaN
    if df.at[i,"standard"] not in ['wysoki','dobry','niski']:
        df.at[i,"standard"] = 'niski' if any(x in df.at[i,"description"] for x in niski_words) else 'dobry'
    
df[['standard']] = df[['standard']].fillna('dobry')
df.standard.value_counts(dropna=False)

In [ ]:
df["city-district"] = df["city"] + "-" + df["district"]

In [ ]:
modes = {}
for x in df["city-district"].unique():
    modes[x] = {}
    for var in ["building_type","building_material","windows","year_built"]:
        try:
            modes[x][var] = df[df["city-district"]==x][var].value_counts().index[0]
        except:
            continue
modes

In [ ]:
for i in tqdm(range(len(df))):
    if df.at[i,"building_material"] not in ['beton', 'pustak', 'cegla', 'inny', 'wielka plyta', 'silikat']:
        try:
            df.at[i,"building_material"] = modes[df.at[i,"city-district"]]["building_material"]
        except:
            continue
    if df.at[i,"building_type"] not in ['blok', 'apartamentowiec', 'kamienica', 'inny', 'szeregowiec']:
        try:
            df.at[i,"building_type"] = modes[df.at[i,"city-district"]]["building_type"]
        except:
            continue
    if df.at[i,"windows"] not in ['plastikowe', 'drewniane', 'aluminiowe']:
        try:
            df.at[i,"windows"] = modes[df.at[i,"city-district"]]["windows"]
        except:
            continue
    if pd.isna(df.at[i,"year_built"]):
        try:
            df.at[i,"year_built"] = modes[df.at[i,"city-district"]]["year_built"]
        except:
            continue
df = df[df["building_material"].isin(['beton', 'pustak', 'cegla', 'inny', 'wielka plyta', 'silikat'])].reset_index(drop=True)
df = df[df["building_type"].isin(['blok', 'apartamentowiec', 'kamienica', 'inny', 'szeregowiec'])].reset_index(drop=True)
df = df[df["windows"].isin(['plastikowe', 'drewniane', 'aluminiowe'])].reset_index(drop=True)

In [ ]:
for x in cat_vars:
    print(x,": ", df[x].value_counts(dropna=False))
    print("--------------------------")

In [ ]:
df = df.dropna(subset=['size', 'floor', 'floors', 'price', 'rooms', 'year_built'])
df.info()

## DEDUPLICATION

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = [str(x.strip().encode('utf-8')) for x in open('polish_stop_words.txt','r').read().split('\n')]
vect = TfidfVectorizer(min_df=1, stop_words=stop_words)

In [ ]:
start=1
data = pd.DataFrame(columns = list(df.columns))
df["unique_apartment"] = ''
for area in df["city-district"].unique():
    df1 = df[df["city-district"]==area].reset_index(drop=True)
    corpus = df1["description"].tolist()
    tfidf = vect.fit_transform(corpus)
    pairwise_similarity = tfidf * tfidf.T
    x = pairwise_similarity.toarray()
    for i in range(len(x)):
        for j in range(len(x[0])):
            if (x[i][j]>0.9) and (
                (df1.at[i,"price"]/df1.at[j,"price"]>0.9) or (df1.at[i,"price"]/df1.at[j,"price"]<1.1)
            ) and (
                (df1.at[i,"size"]/df1.at[j,"size"]>0.85) or (df1.at[i,"size"]/df1.at[j,"size"]<1.15)
            ):
                if (df1.at[i,"unique_apartment"]!='') and (df1.at[j,"unique_apartment"]==''):
                    df1.at[j,"unique_apartment"]=df1.at[i,"unique_apartment"]
                elif (df1.at[j,"unique_apartment"]!='') and (df1.at[i,"unique_apartment"]==''):
                    df1.at[i,"unique_apartment"]=df1.at[j,"unique_apartment"]
                elif (df1.at[j,"unique_apartment"]!='') and (df1.at[i,"unique_apartment"]!=''):
                    continue
                else:
                    df1.at[i,"unique_apartment"] = start
                    df1.at[j,"unique_apartment"] = start
                    start+=1
    data = pd.concat([data,df1], ignore_index=True)
print("offers",df.shape[0])
print("offers2",data.shape[0])
print("unique",len(data["unique_apartment"].unique()))

In [ ]:
for city in data["city"].unique():
    print(city, len(set(data[data["city"]==city]["unique_apartment"].tolist())))

## REMOVING ROWS WITH "STRANGE" VALUES

In [ ]:
data = data[(data["floor"] >= 0) & (data["floor"] <= 50)]
data = data[(data["floors"] >= 0) & (data["floors"] <= 50)]
data = data[(data["price"] >= 500) & (data["price"] <= 25000)]
data = data[(data["rooms"] >= 1) & (data["rooms"] <= 10)]
data = data[(data["size"] >= 10) & (data["size"] <= 200)]
data = data[(data["year_built"] >= 1850) & (data["year_built"] <= 2021)]
data = data[data["city"] != "bydgoszcz"]
data.shape

In [ ]:
len(data["unique_apartment"].unique())

In [ ]:
data1 = data.groupby('unique_apartment').apply(lambda x: x.sample(1)).reset_index(drop=True)
len(data)

In [ ]:
a = data.groupby('unique_apartment').size()
a.value_counts().sort_index()

In [ ]:
columns = ["id","floor","floors","latitude","longitude","price","rooms","size","year_built","building_material","building_type","windows","city","district","balcony","furnished","parking","elevator","separate_kitchen","storage","internet","tv","standard"]
for col in ["balcony","furnished","parking","elevator","separate_kitchen","storage","internet","tv"]:
    data1[col] = data1[col].astype(bool)
data1[columns].to_csv("data_xddd.csv", index=False)

In [ ]:
data1[columns].to_csv("data.csv", index=False)